<a href="https://colab.research.google.com/github/lcrojanouninorte/inndico_sources/blob/main/Indicadores_CTINU_URLS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [48]:
import os
import ssl
import glob
import warnings
import numpy as np
import pandas as pd
import urllib.request
import matplotlib.pyplot as plt
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score

#Folder names
processed_path = os.path.join('..', 'processed')
raw_path = os.path.join('..', 'raw')
warnings.filterwarnings('ignore')
ssl._create_default_https_context = ssl._create_unverified_context
try:
  os.mkdir(f'{raw_path}')
  os.mkdir(f'{processed_path}')
  print('Raw creado')
except OSError as e:
  try:
    os.mkdir(f'{processed_path}')
  except OSError as e:
    print('processed creado')
    

In [49]:
Data_source = [
{
'nombre':'Número de habitantes de Córdoba',
'descripcion':'Indica la cantidad total de personas quehabitan en el departamento',
'url':'https://www.dane.gov.co/files/censo2018/proyecciones-de-poblacion/Departamental/DCD-area-sexo-edad-proyepoblacion-dep-2020-2050-ActPostCOVID-19.xlsx',
'funcion':'procesar_Poblacion'
},
{
'nombre':'PIB a precios constantes',
'descripcion':'Indica el producto interno bruto del departamento a precios constantes',
'url':'https://www.dane.gov.co/files/investigaciones/pib/departamentales/B_2015/anexo-2020-2021-provisional-PIB-total-por-departamento.xlsx',
'funcion':'procesar_PIB'
},
{
'nombre':'Participación en el PIB nacional 2020 - 2021',
'descripcion':'Indica el porcentaje de producto interno bruto del departamento con respecto al total nacional',
'url':'https://www.dane.gov.co/files/investigaciones/pib/departamentales/B_2015/anexo-2020-2021-provisional-PIB-total-por-departamento.xlsx',
'funcion':'procesar_'
},
{
'nombre':'Densidad poblacional de Córdoba (habitantes por kilómetro cuadrado)',
'descripcion':'Indica la utilización del espacio por parte de la población en términos de la cantidad de personas que habitan por cada kilómetro cuadrado del departamento',
'url':'https://www.dane.gov.co/files/censo2018/proyecciones-de-poblacion/Departamental/DCD-area-sexo-edad-proyepoblacion-dep-2020-2050-ActPostCOVID-19.xlsx',
'funcion':'procesar_'
},
{
'nombre':'Densidad poblacional de Córdoba (habitantes por kilómetro cuadrado)',
'descripcion':'Indica la utilización del espacio por parte de la población en términos de la cantidad de personas que habitan por cada kilómetro cuadrado del departamento',
'url':'https://www.dane.gov.co/files/censo2018/proyecciones-de-poblacion/Departamental/DCD-area-sexo-edad-proyepoblacion-dep-2020-2050-ActPostCOVID-19.xlsx',
'funcion':'procesar_'
},
{
'nombre':'Porcentaje de la población en zonas rurales',
'descripcion':'Indica el porcentaje de personas que viven en las zonas rurales del departamento respecto al total de habitantes.',
'url':'https://www.dane.gov.co/files/censo2018/proyecciones-de-poblacion/Departamental/DCD-area-sexo-edad-proyepoblacion-dep-2020-2050-ActPostCOVID-19.xlsx',
'funcion':'procesar_'
},
{
'nombre':'Índice Departamental de Competitividad (IDC) Puntaje, Posición y Grupo de Desempeño',
'descripcion':'Indica el valor y la posición del departamento en el Índice Departamental de Competitividad.',
'url':'https://colaboracion.dnp.gov.co/CDT/Desarrollo%20Empresarial/IDIC/2021/Consolidado_Historico_IDIC_2016_2021.xlsx',
'funcion':'procesar_idc'
},
{
'nombre':'Participación de Córdoba en las exportaciones e importaciones nacionales excluyendo petróleo y sus derivados.',
'descripcion':'Indica el porcentaje de participación que tiene el departamento en las exportaciones e importaciones del país, sin incluir petróleo y sus derivados',
'url':'https://www.dane.gov.co/files/investigaciones/boletines/exportaciones/anexos_exportaciones_feb23.xls',
'funcion':'procesar_'
},
{
'nombre':'Cabezas de ganado sacrificadas',
'descripcion':'Indica el número de cabezas de ganado sacrificadas en Córdoba',
'url':'https://www.dane.gov.co/files/investigaciones/boletines/sacrificio/anexo_ESAG_IVtrim22.xls',
'funcion':'procesar_ESAG'
},
]

In [50]:
Data_source

[{'nombre': 'Número de habitantes de Córdoba',
  'descripcion': 'Indica la cantidad total de personas quehabitan en el departamento',
  'url': 'https://www.dane.gov.co/files/censo2018/proyecciones-de-poblacion/Departamental/DCD-area-sexo-edad-proyepoblacion-dep-2020-2050-ActPostCOVID-19.xlsx',
  'funcion': 'procesar_Poblacion'},
 {'nombre': 'PIB a precios constantes',
  'descripcion': 'Indica el producto interno bruto del departamento a precios constantes',
  'url': 'https://www.dane.gov.co/files/investigaciones/pib/departamentales/B_2015/anexo-2020-2021-provisional-PIB-total-por-departamento.xlsx',
  'funcion': 'procesar_PIB'},
 {'nombre': 'Participación en el PIB nacional 2020 - 2021',
  'descripcion': 'Indica el porcentaje de producto interno bruto del departamento con respecto al total nacional',
  'url': 'https://www.dane.gov.co/files/investigaciones/pib/departamentales/B_2015/anexo-2020-2021-provisional-PIB-total-por-departamento.xlsx',
  'funcion': 'procesar_'},
 {'nombre': 'Den

In [51]:
datos_df = pd.DataFrame(Data_source)
datos_df

,nombre,descripcion,url,funcion
0,Número de habitantes de Córdoba,Indica la cantidad total de personas quehabita...,https://www.dane.gov.co/files/censo2018/proyec...,procesar_Poblacion
1,PIB a precios constantes,Indica el producto interno bruto del departame...,https://www.dane.gov.co/files/investigaciones/...,procesar_PIB
2,Participación en el PIB nacional 2020 - 2021,Indica el porcentaje de producto interno bruto...,https://www.dane.gov.co/files/investigaciones/...,procesar_
3,Densidad poblacional de Córdoba (habitantes po...,Indica la utilización del espacio por parte de...,https://www.dane.gov.co/files/censo2018/proyec...,procesar_
4,Densidad poblacional de Córdoba (habitantes po...,Indica la utilización del espacio por parte de...,https://www.dane.gov.co/files/censo2018/proyec...,procesar_
5,Porcentaje de la población en zonas rurales,Indica el porcentaje de personas que viven en ...,https://www.dane.gov.co/files/censo2018/proyec...,procesar_
6,Índice Departamental de Competitividad (IDC) P...,Indica el valor y la posición del departamento...,https://colaboracion.dnp.gov.co/CDT/Desarrollo...,procesar_idc
7,Participación de Córdoba en las exportaciones ...,Indica el porcentaje de participación que tien...,https://www.dane.gov.co/files/investigaciones/...,procesar_
8,Cabezas de ganado sacrificadas,Indica el número de cabezas de ganado sacrific...,https://www.dane.gov.co/files/investigaciones/...,procesar_ESAG


In [52]:
def delete_files_old():
  with os.scandir(f'{raw_path}') as ficheros:
    for fichero in ficheros:
      if fichero.name not in ['.config','sample_data']:
        try:
          os.remove(fichero.name)
        except OSError as e:
          print(f"Error:{ e.strerror}")
def downolad_data(datos_df):
  delete_files_old()
  datos_df['nombre_archivo'] = ""
  for i in range(len(datos_df)):
    try:
      file_name = datos_df['url'][i]
      urllib.request.urlretrieve(file_name, os.path.join(raw_path, file_name.split('/')[-1]))
      datos_df['nombre_archivo'][i] =  os.path.join(raw_path, file_name.split('/')[-1])
    except OSError as e:
      datos_df['nombre_archivo'][i] = 'Error'
  return datos_df
datos_df = downolad_data(datos_df)
datos_df

Error:No such file or directory
Error:No such file or directory
Error:No such file or directory
Error:No such file or directory
Error:No such file or directory


,nombre,descripcion,url,funcion,nombre_archivo
0,Número de habitantes de Córdoba,Indica la cantidad total de personas quehabita...,https://www.dane.gov.co/files/censo2018/proyec...,procesar_Poblacion,../raw/DCD-area-sexo-edad-proyepoblacion-dep-2...
1,PIB a precios constantes,Indica el producto interno bruto del departame...,https://www.dane.gov.co/files/investigaciones/...,procesar_PIB,../raw/anexo-2020-2021-provisional-PIB-total-p...
2,Participación en el PIB nacional 2020 - 2021,Indica el porcentaje de producto interno bruto...,https://www.dane.gov.co/files/investigaciones/...,procesar_,../raw/anexo-2020-2021-provisional-PIB-total-p...
3,Densidad poblacional de Córdoba (habitantes po...,Indica la utilización del espacio por parte de...,https://www.dane.gov.co/files/censo2018/proyec...,procesar_,../raw/DCD-area-sexo-edad-proyepoblacion-dep-2...
4,Densidad poblacional de Córdoba (habitantes po...,Indica la utilización del espacio por parte de...,https://www.dane.gov.co/files/censo2018/proyec...,procesar_,../raw/DCD-area-sexo-edad-proyepoblacion-dep-2...
5,Porcentaje de la población en zonas rurales,Indica el porcentaje de personas que viven en ...,https://www.dane.gov.co/files/censo2018/proyec...,procesar_,../raw/DCD-area-sexo-edad-proyepoblacion-dep-2...
6,Índice Departamental de Competitividad (IDC) P...,Indica el valor y la posición del departamento...,https://colaboracion.dnp.gov.co/CDT/Desarrollo...,procesar_idc,../raw/Consolidado_Historico_IDIC_2016_2021.xlsx
7,Participación de Córdoba en las exportaciones ...,Indica el porcentaje de participación que tien...,https://www.dane.gov.co/files/investigaciones/...,procesar_,../raw/anexos_exportaciones_feb23.xls
8,Cabezas de ganado sacrificadas,Indica el número de cabezas de ganado sacrific...,https://www.dane.gov.co/files/investigaciones/...,procesar_ESAG,../raw/anexo_ESAG_IVtrim22.xls


In [64]:
def procesar_idc(fuente):
  lista_indicadores = [['INS11', 'ENTORNO POLÍTICO'],['INS111','ÍNDICE DE ESTABILIDAD POLÍTICA Y AUSENCIA DE VIOLENCIA / TERRORISMO'],['INS112', 'ÍNDICE DE EFECTIVIDAD DEL GOBIERNO'],['INS113', 'VIOLACIONES A LA LIBERTAD DE PRENSA'],['INS12', 'ENTORNO REGULATORIO'], ['INS121','CAPACIDAD DE IMPLEMENTACIÓN DE POLÍTICAS Y REGULACIONES'],['INS122', 'ÍNDICE DE PRIMACÍA DE LA LEY'],['INS123', 'PROPORCIÓN DE OCUPADOS CON SEGURIDAD SOCIAL '],['INS13', 'AMBIENTE DE NEGOCIOS'],['INS131', 'ÍNDICE DE APERTURA DE NEGOCIO'], ['INS132', 'ÍNDICE DE PAGOS DE IMPUESTOS POR AÑO'],['INS133', 'ÍNDICE DE REGISTRO DE PROPIEDADES'],['CHI21', 'EDUCACIÓN SECUNDARIA Y MEDIA'],['CHI211', 'TASA DE COBERTURA NETA EN EDUCACIÓN SECUNDARIA'],['CHI212', 'TASA DE COBERTURA NETA EN EDUCACIÓN MEDIA'], ['CHI213', 'ESPERANZA DE VIDA ESCOLAR'],['CHI214','EVALUACIÓN DEL DESEMPEÑO ESCOLAR EN LECTURA, MATEMÁTICAS Y CIENCIA  EN LAS PRUEBAS SABER 11'], ['CHI22', 'EDUCACIÓN SUPERIOR'],['CHI221', 'TASA DE COBERTURA EN EDUCACIÓN SUPERIOR'], ['CHI222','ÍNDICE DE PARTICIPACIÓN DE GRADUADOS EN CIENCIAS E INGENIERÍAS  STEM'], ['CHI223', 'MOVILIDAD INTERNACIONAL (ÍNDICE) '],['CHI23', 'INVESTIGACIÓN Y DESARROLLO'], ['CHI231', 'NÚMERO DE INVESTIGADORES POR MILLÓN DE HABITANTES'], ['CHI232','GASTO BRUTO EN INVESTIGACIÓN Y DESARROLLO (I+D) COMO PORCENTAJE DEL PIB'], ['CHI233','DESEMPEÑO DE LOS ESTUDIANTES DE EDUCACION TERCIARIA EN LAS PRUEBAS DE ESTADO SABER'], ['CHI234', 'INTENSIDAD DEL GASTO EMPRESARIAL EN I+D (PORCENTAJE)'], ['INF31', 'TIC'],['INF311', 'ACCESO A LAS TIC'], ['INF312', 'USO DE LAS TIC'], ['INF313', 'SERVICIOS DIGITALES DE CONFIANZA Y SEGURIDAD '], ['INF314', 'GOBIERNO DIGITAL'], ['INF32', 'INFRAESTRUCTURA GENERAL'], ['INF321', 'ÍNDICE DE GENERACIÓN Y COSTO DE LA ENERGÍA'], ['INF322', 'DESEMPEÑO LOGÍSTICO '], ['INF323', 'INVERSIÓN PÚBLICA EN CAPITAL FIJO  (% DEL PIB)'], ['INF33', 'SOSTENIBILIDAD ECOLÓGICA'], ['INF331', 'EFICIENCIA EN EL USO DE LA ENERGÍA (EN UNIDADES DEL PIB) '], ['INF332', 'DESEMPEÑO AMBIENTAL '], ['INF333', 'EMPRESAS CON CERTIFICACIÓN AMBIENTAL ISO 14001 '], ['SOFM41', 'CRÉDITO'], ['SOFM411', 'FACILIDAD PARA OBTENER CRÉDITO'], ['SOFM412', 'CRÉDITO DOMÉSTICO '], ['SOFM413', 'CRÉDITO A LAS EMPRESAS PARA INNOVAR '], ['SOFM42', 'CAPACIDAD DE INVERSIÓN'], ['SOFM421', 'CAPITALIZACIÓN BURSÁTIL '], ['SOFM422','INVERSIÓN PRIVADA EN CAPITAL FIJO EN LA INDUSTRIA (% DEL PIB) '], ['SOFM423','INVERSIÓN EN ACTIVIDADES CONDUCENTES A LA INNOVACIÓN EN LAS EMPRESAS (% DEL PIB) '], ['SOFM43', 'COMERCIO Y COMPETENCIA'], ['SOFM431', 'DIVERSIFICACIÓN DE MERCADOS DE DESTINO DE EXPORTACIONES '], ['SOFM432','DIVERSIFICACIÓN DE LA CANASTA EXPORTADORA (ÍNDICE HERFINDAHLHIRSCHMAN) '], ['SOFM433', 'SOFISTICACIÓN DEL APARATO PRODUCTIVO '], ['SOFM434', 'TAMAÑO DEL MERCADO INTERNO (LOGARITMO)'], ['SOFN51', 'TRABAJADORES DE CONOCIMIENTO'], ['SOFN511', 'EMPLEO INTENSIVO EN CONOCIMIENTO (%)'],['SOFN512','GASTO EN FORMACIÓN Y CAPACITACIÓN (POR CADA 100 MIL UNIDADES DEL PIB)'], ['SOFN513', 'GASTO PRIVADO EN I+D (% DEL PIB)'],['SOFN514', 'FINANCIACIÓN EMPRESARIAL DEL GASTO EN I+D (%)'],['SOFN515','BECAS DE MAESTRÍA Y DOCTORADO POR CADA 100 MIL PERSONAS ENTRE 25 Y 40 AÑOS'],['SOFN516', 'MUJERES EMPLEADAS CON FORMACIÓN DE ALTO NIVEL '], ['SOFN52', 'ENLACES DE INNOVACIÓN'], ['SOFN521','EMPRESAS QUE COOPERAN EN ACTIVIDADES DE INNOVACIÓN CON ORGANIZACIONES DEL CONOCIMIENTO (%)'], ['SOFN522', 'ÍNDICE DE ESPECIALIZACIÓN INDUSTRIAL'],['SOFN523', 'GASTO EN I+D FINANCIADO DESDE EL EXTRANJERO (%)'], ['SOFN524', 'EMPRESAS QUE COOPERAN CON ORGANIZACIONES INTERNACIONALES (%)'],['SOFN53', 'ABSORCIÓN DEL CONOCIMIENTO'],['SOFN531','INVERSIÓN EN TRANSFERENCIA DE TECNOLOGÍA (POR CADA 100 MIL UNIDADES DEL PIB)'],['SOFN532','IMPORTACIONES DE BIENES DE ALTA TECNOLOGÍA (% DE LAS IMPORTACIONES TOTALES DEL DEPARTAMENTO)'],['SOFN533', 'INVERSIÓN EXTRANJERA DIRECTA DESDE EL EXTERIOR'],['SOFN534', 'PERSONAL EN ACTI POR CADA 100 MIL HABITANTES'], ['PCT61', 'CREACIÓN DE CONOCIMIENTO'],['PCT611', 'SOLICITUDES DE PATENTES A RESIDENTES POR MILLÓN DE HABITANTES'],['PCT612', 'EMPRESAS INNOVADORAS EN SENTIDO ESTRICTO (%)'],['PCT613','SOLICITUDES DE MODELOS DE UTILIDAD REALIZADAS POR RESIDENTES POR MILLÓN DE HABITANTES'], ['PCT614', 'ARTÍCULOS EN PUBLICACIONES CIENTÍFICAS Y TÉCNICAS '],['PCT615', 'ÍNDICE H (HIRSCH) DE DOCUMENTOS CITABLES'],['PCT616', 'EMPRESAS INNOVADORAS EN SENTIDO AMPLIO (%)'],['PCT62', 'IMPACTO DEL CONOCIMIENTO'],['PCT621', 'PIB POR TRABAJADOR (MILLONES DE PESOS) '],['PCT622', 'TASA DE NATALIDAD EMPRESARIAL NETA POR CADA 10 MIL HABITANTES'], ['PCT623','GASTO EN TIC PARA INNOVAR (TASA POR CADA 10 MIL UNIDADES DEL PIB)'],['PCT624', 'CERTIFICADOS DE CALIDAD ISO DE PRODUCTOS Y DE PROCESOS (%)'],['PCT625','PRODUCCIÓN DE ALTO Y MEDIO ALTO CONTENIDO TECNOLÓGICO EN LA MANUFACTURA (%)'], ['PCT626', 'VENTAS POR INNOVAR'],['PCT63', 'DIFUSIÓN DE CONOCIMIENTO'],['PCT631', 'EXPORTACIONES DE PRODUCTOS ALTA TECNOLOGÍA (%)'],['PCT632','EMPRESAS DE SERVICIOS DE COMUNICACIÓN, COMPUTACIÓN E INFORMACIÓN QUE EXPORTAN (%)'], ['PCT633','PATENTES CONCEDIDAS EN LOS ÚLTIMOS TRES AÑOS POR MILLÓN DE HABITANTES'],['PCR71', 'BIENES INTANGIBLES'],['PCR711', 'SOLICITUDES DE REGISTRO DE MARCAS POR MILLÓN DE HABITANTES'],['PCR712', 'TIC Y CREACIÓN DE NUEVOS MODELOS DE NEGOCIOS (% DE EMPRESAS)'], ['PCR713','INVERSIÓN EN TIC DE LAS EMPRESAS QUE INTRODUCEN NUEVOS MÉTODOS ORGANIZATIVOS (TASA POR CADA MILLÓN DE UNIDADES DEL PIB)'],['PCR714','SOLICITUDES DE REGISTRO DE DISEÑOS INDUSTRIALES POR MILLÓN DE HABITANTES '],['PCR72', 'BIENES Y SERVICIOS CREATIVOS'], ['PCR721', 'EXPORTACIÓN DE SERVICIOS CREATIVOS Y CULTURALES (%)'],['PCR722', 'PRODUCCIÓN DE ECONOMÍA NARANJA (% PIB DEL DEPARTAMENTO)'],['PCR73', 'CREATIVIDAD EN LÍNEA'],['PCR731','SOLICITUDES DE PATENTES  EN EL SECTOR TIC POR MILLÓN DE HABITANTES'],['PCR732', 'EMPRENDIMIENTOS DIGITALES '],['PCR733', 'REGISTROS DE SOFTWARE']]
  code_indicadores = pd.DataFrame(lista_indicadores)
  code_indicadores = code_indicadores.rename(columns={0: "code_i", 1: "descripcion"})
  df = pd.read_excel(fuente).query("DEPARTAMENTO=='CÓRDOBA'")
  df = df.drop(['DEPARTAMENTO'],axis=1)
  df = df.rename(columns={'IDIC':'code_i'})
  df.index = df.code_i
  df = df.sort_index()
  df = df.T
  df['code'] = df.index 
  df = pd.merge(df, code_indicadores, how='inner', left_on='code', right_on='code_i')
  df = df.drop(['code','code_i'],axis=1)
  data = []
  for i in df:
    if str(i).__contains__('20'):
      for j in range(len(df)):
        data.append([i,df[i][j],j,df['descripcion'][j]])
  df = pd.DataFrame(data)
  df = df.drop([2],axis=1)
  df = df.rename(columns={0:"Ano",1:"Puntaje",3:'Indicador'})
  df.to_csv(f'{processed_path}/Procesado_IDIC_puntaje.csv')
  df = pd.read_excel(fuente,sheet_name="Posición").query("DEPARTAMENTO=='CÓRDOBA'")
  df = df.drop(['DEPARTAMENTO'],axis=1)
  df = df.rename(columns={'IDIC':'code_i'})
  df.index = df.code_i
  df = df.sort_index()
  df = df.T
  df['code'] = df.index 
  df = pd.merge(df, code_indicadores, how='inner', left_on='code', right_on='code_i')
  df = df.drop(['code','code_i'],axis=1)
  data = []
  for i in df:
    if str(i).__contains__('20'):
      for j in range(len(df)):
        data.append([i,df[i][j],j,df['descripcion'][j]])
  df = pd.DataFrame(data)
  df = df.drop([2],axis=1)
  df = df.rename(columns={0:"Ano",1:"Pocision",3:'Indicador'})
  df.to_csv(f'{processed_path}/Procesado_IDIC_pocision.csv')
  df = pd.read_excel(fuente,sheet_name="Grupo de Desempeño").query("DEPARTAMENTO=='CÓRDOBA'")
  df = df.drop(['DEPARTAMENTO'],axis=1)
  df = df.rename(columns={'IDIC':'code_i'})
  df.index = df.code_i
  df = df.sort_index()
  df = df.T
  df['code'] = df.index 
  df = pd.merge(df, code_indicadores, how='inner', left_on='code', right_on='code_i')
  df = df.drop(['code','code_i'],axis=1)
  data = []
  for i in df:
    if str(i).__contains__('20'):
      for j in range(len(df)):
        data.append([i,df[i][j],j,df['descripcion'][j]])
  df = pd.DataFrame(data)
  df = df.drop([2],axis=1)
  df = df.rename(columns={0:"Ano",1:"Grupo_Desempeno",3:'Indicador'})
  df.to_csv(f'{processed_path}/Procesado_IDIC_Grupo_Desempeno.csv', index=False)

In [65]:
def procesar_ESAG(fuente):
  df = pd.read_excel(fuente,sheet_name="Anexo3")
  df = df.rename(columns={'Unnamed: 0':"Departamento",
                          'Unnamed: 1':"IV trim 2021 Número de cabezas",
                          'Unnamed: 2':'IV trim 2022 Número de cabezas',
                          'Unnamed: 3':"Participación % IV trim 2022",
                          'Unnamed: 4':'Variación % 2022/2021',
                          'Unnamed: 5':'Contribución',
                          })
  df = df.query("Departamento=='Córdoba'")
  df = df.drop(['Unnamed: 6'],axis=1)
  df.to_csv(f'{processed_path}/Procesado_Participacion_variacion_contribucion_sacrificio_ganado_vacuno.csv')
  df = pd.read_excel(fuente,sheet_name="Anexo6")
  df = df.rename(columns={'Unnamed: 0':"Departamento",
                          'Unnamed: 1':"Peso en pie (t) 2021",
                          'Unnamed: 2':'Peso promedio  en pie (kg) 2021',
                          'Unnamed: 3':"Peso en pie (t) 2022",
                          'Unnamed: 4':'Peso promedio  en pie (kg) 2022',
                          'Unnamed: 5':'Variación % 2022/2021 peso en pie',
                          'Unnamed: 6':'Variación % 2022/2021 peso promedio',
                          })
  df = df.query("Departamento=='Córdoba'")
  df = df.drop(['Unnamed: 7','Unnamed: 8'],axis=1)
  df.to_csv(f'{processed_path}/Procesado_Peso_pie_ganado_vacuno.csv')
  df = pd.read_excel(fuente,sheet_name="Anexo10")
  df = df.rename(columns={'Unnamed: 0':"Departamento",
                          'Unnamed: 1':"Peso en canal (t) 2021",
                          'Unnamed: 2':'Peso promedio  en canal % 2021',
                          'Unnamed: 4':"Peso en canal (t) 2022",
                          'Unnamed: 5':'Peso promedio  en canal % 2022',
                          'Unnamed: 6':'Variación % 2022/2021 peso en canal',
                          'Unnamed: 7':'Variación % 2022/2021 rendimiento de la canal',
                          })
  df = df.query("Departamento=='Córdoba'")
  df = df.drop(['Unnamed: 3','Unnamed: 8'],axis=1)
  df.to_csv(f'{processed_path}/Procesado_Peso_canal_rendimiento_canal_ganado_vacuno.xlsx')
  df = pd.read_excel(fuente,sheet_name="Anexo12")
  df = df.rename(columns={'Unnamed: 0':"Departamento",
                          'Unnamed: 1':"Cabezas de ganado",
                          'Unnamed: 2':'Participación %',
                          })
  df = df.query("Departamento=='Córdoba'")
  df = df.drop(['Unnamed: 3'],axis=1)
  df.to_csv(f'{processed_path}/Procesado_Procedencia_ganado_vacuno.csv')
  df = pd.read_excel(fuente,sheet_name="Anexo13")
  df = df.rename(columns={'Unnamed: 0':"Departamento",
                          'Unnamed: 1':"Cabezas de ganado",
                          'Unnamed: 2':'Participación %',
                          })
  df = df.query("Departamento=='Córdoba'")
  df = df.drop(['Unnamed: 3'],axis=1)
  df.to_csv(f'{processed_path}/Procesado_Procedencia_ganado_bufalino.csv', index=False)

In [66]:

def procesar_PIB(fuente):
  df = pd.read_excel(f'{raw_path}/anexo-2020-2021-provisional-PIB-total-por-departamento.xlsx',sheet_name="Cuadro 1",skiprows=9).query("DEPARTAMENTOS=='Córdoba'")
  df = df.drop(['Código Departamento (DIVIPOLA)','DEPARTAMENTOS'],axis=1)
  df = df[df.columns[0:17]]
  df = df.rename(columns={'2020p':"2020",'2021p':"2021" })
  df = df.T
  df = df.rename(columns={14:"Producto Interno Bruto por departamento A precios corrientes"})
  df.to_csv(f'{processed_path}/Procesado_Producto_Interno_Bruto_departamento_precios_corrientes.csv')
  df = pd.read_excel(f'{raw_path}/anexo-2020-2021-provisional-PIB-total-por-departamento.xlsx',sheet_name="Cuadro 2",skiprows=9).query("DEPARTAMENTOS=='Córdoba'")
  df = df.drop(['Código Departamento (DIVIPOLA)','DEPARTAMENTOS'],axis=1)
  df = df[df.columns[0:17]]
  df = df.rename(columns={'2020p':"2020",'2021p':"2021" })
  df = df.T
  df = df.rename(columns={14:"Series encadenadas de volumen con año de referencia"})
  df.to_csv(f'{processed_path}/Procesado_Series_encadenadas_volumen_año_referencia.csv')
  df = pd.read_excel(f'{raw_path}/anexo-2020-2021-provisional-PIB-total-por-departamento.xlsx',sheet_name="Cuadro 3",skiprows=8).query("DEPARTAMENTOS=='Córdoba'")
  df = df.drop(['Código Departamento (DIVIPOLA)','DEPARTAMENTOS'],axis=1)
  df = df[df.columns[0:17]]
  df = df.rename(columns={'2020p':"2020",'2021p':"2021" })
  df = df.T
  df = df.rename(columns={14:"Producto Interno Bruto departamental por habitante a precios corrientes"})
  df.to_csv(f'{processed_path}/Procesado_Producto_Interno_Bruto_habitante_precios_corrientes.csv', index=False)

In [62]:
def procesar_Poblacion(fuente):
  df = pd.read_excel(fuente,skiprows=11,dtype=str).query("DPNOM=='Córdoba'")
  df = df.rename(columns={'ÁREA GEOGRÁFICA':"AREAGEOGRAFICA"})
  df.query("AREAGEOGRAFICA!='Total'")
  df = df.drop(['DP','DPNOM'],axis=1)
  df.index = df['AÑO']
  df = df.drop(['AÑO'],axis=1)
  df = df.sort_index()
  df.to_csv(f'{processed_path}/Procesado_Poblacion.csv', index=False)

In [67]:
def Analizar(datos_df):
  for i in range(len(datos_df)):
    if datos_df['funcion'][i]=='procesar_idc':
      procesar_idc(datos_df['nombre_archivo'][i])
    if datos_df['funcion'][i]=='procesar_ESAG':
      procesar_ESAG(datos_df['nombre_archivo'][i])
    if datos_df['funcion'][i]=='procesar_PIB':
      procesar_PIB(datos_df['nombre_archivo'][i])
    if datos_df['funcion'][i]=='procesar_Poblacion':
      procesar_Poblacion(datos_df['nombre_archivo'][i])
  
Analizar(datos_df)

In [ ]:
df = pd.read_csv(f'{processed_path}/Procesado_IDIC_pocision.csv')
df = df.drop(['Unnamed: 0'],axis=1)
df = df.dropna()
for i in df.Indicador.unique():
  d = df.query(f'Indicador == "%s"'%i) #.to_csv(f'procesado/idic_%s.csv'%i.replace(' ','_')[0:30], index=False)
  dataX = d[["Ano"]]
  X_train = np.array(dataX)
  y_train = d['Pocision'].values
  proyeccion = []
  for p in range(4):
    proyeccion.append([X_train[-1][0]+1+p])
  regr = linear_model.LinearRegression()
  regr.fit(X_train, y_train)
  y_pred = regr.predict(proyeccion)
  d = d.reset_index()
  d = d.drop(['index'],axis=1)
  modelo = []
  for m in range(len(proyeccion)):
    modelo.append([proyeccion[m][0],y_pred[m],'Prediccion'])
  predic = pd.DataFrame(modelo)
  predic.rename(columns={0:'Ano',1:'Pocision',2:'Indicador'},inplace=True)
  all_datos = pd.concat([d,predic], axis=0)
  all_datos.to_csv(f'{processed_path}/idic_%s.csv'%i.replace(' ','_')[0:30], index=False)